In [ ]:
import chess as ch
import random as rd

class Engine:

    def __init__(self, board, maxDepth, color):
        self.board=board
        self.color=color
        self.maxDepth=maxDepth
    
    def getBestMove(self):
        return self.engine(None, 1)

    def evalFunct(self):
        compt = 0
        #Sums up the material values
        for i in range(64):
            compt+=self.squareResPoints(ch.SQUARES[i])
        compt += self.mateOpportunity() + self.openning() + 0.001*rd.random()
        return compt

    def mateOpportunity(self):
        if (self.board.legal_moves.count()==0):
            if (self.board.turn == self.color):
                return -999
            else:
                return 999
        else:
            return 0

    #to make the engine developp in the first moves
    def openning(self):
        if (self.board.fullmove_number<10):
            if (self.board.turn == self.color):
                return 1/20 * self.board.legal_moves.count()
            else:
                return -1/20 * self.board.legal_moves.count()
        else:
            return 0

    #Takes a square as input and 
    #returns the corresponding Hans Berliner's
    #system value of it's resident
    def squareResPoints(self, square):
        pieceValue = 0
        if(self.board.piece_type_at(square) == ch.PAWN):
            pieceValue = 1
        elif (self.board.piece_type_at(square) == ch.ROOK):
            pieceValue = 5.1
        elif (self.board.piece_type_at(square) == ch.BISHOP):
            pieceValue = 3.33
        elif (self.board.piece_type_at(square) == ch.KNIGHT):
            pieceValue = 3.2
        elif (self.board.piece_type_at(square) == ch.QUEEN):
            pieceValue = 8.8

        if (self.board.color_at(square)!=self.color):
            return -pieceValue
        else:
            return pieceValue

        
    def engine(self, candidate, depth):
        
        #reached max depth of search or no possible moves
        if ( depth == self.maxDepth
        or self.board.legal_moves.count() == 0):
            return self.evalFunct()
        
        else:
            #get list of legal moves of the current position
            moveListe = list(self.board.legal_moves)
            
            #initialise newCandidate
            newCandidate = None
            #(uneven depth means engine's turn)
            if(depth % 2 != 0):
                newCandidate = float("-inf")
            else:
                newCandidate = float("inf")
            
            #analyse board after deeper moves
            for i in moveListe:

                #Play move i
                self.board.push(i)

                #Get value of move i (by exploring the repercussions)
                value = self.engine(newCandidate, depth + 1) 

                #Basic minmax algorithm:
                #if maximizing (engine's turn)
                if(value > newCandidate and depth % 2 != 0):
                    #need to save move played by the engine
                    if (depth == 1):
                        move=i
                    newCandidate = value
                #if minimizing (human player's turn)
                elif(value < newCandidate and depth % 2 == 0):
                    newCandidate = value

                #Alpha-beta prunning cuts: 
                #(if previous move was made by the engine)
                if (candidate != None
                 and value < candidate
                 and depth % 2 == 0):
                    self.board.pop()
                    break
                #(if previous move was made by the human player)
                elif (candidate != None 
                and value > candidate 
                and depth % 2 != 0):
                    self.board.pop()
                    break
                
                #Undo last move
                self.board.pop()

            #Return result
            if (depth>1):
                #eturn value of a move in the tree
                return newCandidate
            else:
                #return the move (only on first move)
                return move





import chess as ch

class Main:

    def __init__(self, board=ch.Board):
        self.board=board

    #play human move
    def playHumanMove(self):
        try:
            print(self.board.legal_moves)
            print("""To undo your last move, type "undo".""")
            #get human move
            play = input("Your move: ")
            if (play=="undo"):
                self.board.pop()
                self.board.pop()
                self.playHumanMove()
                return
            self.board.push_san(play)
        except:
            self.playHumanMove()

    #play engine move
    def playEngineMove(self, maxDepth, color):
        engine = Engine(self.board, maxDepth, color)
        self.board.push(engine.getBestMove())

    #start a game
    def startGame(self):
        #get human player's color
        color=None
        while(color!="b" and color!="w"):
            color = input("""Play as (type "b" or "w"): """)
        maxDepth=None
        while(isinstance(maxDepth, int)==False):
            maxDepth = int(input("""Choose depth: """))
        if color=="b":
            while (self.board.is_checkmate()==False):
                print("The engine is thinking...")
                self.playEngineMove(maxDepth, ch.WHITE)
                print(self.board)
                self.playHumanMove()
                print(self.board)
            print(self.board)
            print(self.board.outcome())    
        elif color=="w":
            while (self.board.is_checkmate()==False):
                print(self.board)
                self.playHumanMove()
                print(self.board)
                print("The engine is thinking...")
                self.playEngineMove(maxDepth, ch.BLACK)
            print(self.board)
            print(self.board.outcome())
        #reset the board
        self.board.reset
        #start another game
        self.startGame()

#create an instance and start a game
newBoard= ch.Board()
game = Main(newBoard)
bruh = game.startGame()

Play as (type "b" or "w"): b
Choose depth: 4
The engine is thinking...
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . P . . .
P P P P . P P P
R N B Q K B N R
<LegalMoveGenerator at 0x7faf30989040 (Nh6, Nf6, Nc6, Na6, h6, g6, f6, e6, d6, c6, b6, a6, h5, g5, f5, e5, d5, c5, b5, a5)>
To undo your last move, type "undo".
Your move: e5
r n b q k b n r
p p p p . p p p
. . . . . . . .
. . . . p . . .
. . . . . . . .
. . . . P . . .
P P P P . P P P
R N B Q K B N R
The engine is thinking...
r n b q k b n r
p p p p . p p p
. . . . . . . .
. . . . p . . Q
. . . . . . . .
. . . . P . . .
P P P P . P P P
R N B . K B N R
<LegalMoveGenerator at 0x7faf309aca90 (Ne7, Nh6, Nf6, Be7, Bd6, Bc5, Bb4, Ba3, Ke7, Qe7, Qf6, Qg5, Qh4, Nc6, Na6, h6, g6, d6, c6, b6, a6, e4, g5, d5, c5, b5, a5)>
To undo your last move, type "undo".
Your move: qf6
<LegalMoveGenerator at 0x7faf309ac280 (Ne7, Nh6, Nf6, Be7, Bd6, Bc5, Bb4, Ba3, Ke7, Qe7, Qf6, Qg5, Qh4, Nc6, Na6, h6, g6, d6, c6,

Your move: Qd5
. . r k . b r .
p . p b n p p .
. . p p . . . p
. . . q p . . .
P P . . . . . Q
N . P . P . P N
. . . P . P . P
. R B . K . . R
The engine is thinking...
. . r k . b r .
p . p b n p p .
. . p p . . . p
. . . q p . . .
P P . . P . . Q
N . P . . . P N
. . . P . P . P
. R B . K . . R
<LegalMoveGenerator at 0x7faf2000c160 (Rh8, Ke8, Rb8, Ra8, Be8, Be6, Bf5, Bg4, Bxh3, Qe6, Qc5, Qb5, Qa5, Qxe4+, Qd4, Qc4, Qd3, Qb3, Qxd2+, Qa2, g6, f6, a6, h5, c5, g5, f5, a5)>
To undo your last move, type "undo".
Your move: Qd3
. . r k . b r .
p . p b n p p .
. . p p . . . p
. . . . p . . .
P P . . P . . Q
N . P q . . P N
. . . P . P . P
. R B . K . . R
The engine is thinking...
. . r k . b r .
p . p b n p p .
. . p p . . . p
. . . . p . . .
P P . . P . . Q
N . P q . . P .
. . . P . P . P
. R B . K . N R
<LegalMoveGenerator at 0x7faf2000cdf0 (Rh8, Ke8, Rb8, Ra8, Be8, Be6, Bf5, Bg4, Bh3, Qa6, Qd5, Qb5, Qxe4+, Qd4, Qc4, Qxg3, Qf3, Qe3+, Qxc3, Qe2+, Qxd2+, Qc2, Qf1+, Qxb1, g6, f6, a6, h5, d5, c5,

Your move: h5
. . r k . . . .
. . . b n Q b r
p . q p . . . .
P . . . p . p p
. N P . P . . .
. . . . . P P .
. . . P K . . P
. R B . . . N R
The engine is thinking...
. . r k . . . .
. . . b n Q b r
p . N p . . . .
P . . . p . p p
. . P . P . . .
. . . . . P P .
. . . P K . . P
. R B . . . N R
<LegalMoveGenerator at 0x7faf2000c910 (Kc7, Rxc6, Nxc6, Bxc6)>
To undo your last move, type "undo".
Your move: bxc6
<LegalMoveGenerator at 0x7faf2000cca0 (Kc7, Rxc6, Nxc6, Bxc6)>
To undo your last move, type "undo".
Your move: Bxc6
. . r k . . . .
. . . . n Q b r
p . b p . . . .
P . . . p . p p
. . P . P . . .
. . . . . P P .
. . . P K . . P
. R B . . . N R
The engine is thinking...
. . r k . . . .
. . . . n . b r
p . b p Q . . .
P . . . p . p p
. . P . P . . .
. . . . . P P .
. . . P K . . P
. R B . . . N R
<LegalMoveGenerator at 0x7faf200085b0 (Ke8, Kc7, Rb8, Ra8, Rc7, Rh8, Rh6, Bh8, Bf8, Bh6, Bf6, Ng8, Ng6, Nf5, Nd5, Be8, Ba8, Bd7, Bb7, Bd5, Bb5, Bxe4, Ba4, d5, h4, g4)>
To undo your last move

In [ ]:
pip install Chess